# Importing Labiraries

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

# Data Preprocessing

### (Read & Separate) samples & labels

In [2]:
# load data
train_data = pd.read_csv('./train.csv')
#separate the target col
train_label = train_data.Survived
#Store the Ids as indcies for the df
train_data.index = train_data['PassengerId']
# Drop unused cols
features_toDrop = ['Survived', 'Ticket', 'Cabin', 'Name', 'PassengerId']
train_data = train_data.drop(features_toDrop, axis = 1)

In [3]:
def create_Fsize_feature(data):
    
    # Create new feature (family size)
    data["Fsize"] = data["SibSp"] + data["Parch"] + 1
    
    # encode the family size feature
    data['Single'] = data['Fsize'].map(lambda s: 1 if s == 1 else 0)
    data['SmallF'] = data['Fsize'].map(lambda s: 1 if  s == 2  else 0)
    data['MedF'] = data['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
    data['LargeF'] = data['Fsize'].map(lambda s: 1 if s >= 5 else 0)
    
    return data

In [4]:
def sex_embark_encode(data):
    
    data['Sex'] = data['Sex'].astype('category')
    data['Sex'] = data['Sex'].cat.codes
    
    data['Embarked'] = data['Embarked'].fillna('S') 
    data['Embarked'] = data['Embarked'].astype('category')
    data['Embarked'] = data['Embarked'].cat.codes
    
    return data

In [5]:
train_data = create_Fsize_feature(train_data)
train_data = sex_embark_encode(train_data)

In [6]:
print("The dataframe columns are:\n")
print(train_data.columns)

The dataframe columns are:

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Fsize',
       'Single', 'SmallF', 'MedF', 'LargeF'],
      dtype='object')


### Data Cleaning

In [7]:
Mean = train_data.loc[:,'Age'].mean()
print("Mean of Age is:", Mean)
train_data['Age'] = train_data['Age'].fillna(Mean) # Fill the nan values with the mean of Age values

Mean of Age is: 29.69911764705882


In [8]:
print("Number of nan values in Age Col is: ",train_data['Age'].isnull().sum()) # Check for nan values in Age

Number of nan values in Age Col is:  0


In [9]:
print(len(train_data))
print("Number of nan values in the whole df:\n", train_data.isnull().sum())

891
Number of nan values in the whole df:
 Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Fsize       0
Single      0
SmallF      0
MedF        0
LargeF      0
dtype: int64


### Encoding 'Embarked & Sex'

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
sc = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size = 0.2, shuffle = True)

sc.fit(X_train)

X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [11]:
#lr = LogisticRegression(random_state = 0)  # Test acc: 0.76076 with scaling
                                           # Test acc: 0.75358 without scaling which is weird for me XD
    
#lr = MLPClassifier(hidden_layer_sizes = [100,10,10], random_state = 0, max_iter = 1000)  # Test acc: 0.7655 with scaling

lr= GradientBoostingClassifier(random_state = 0)
lr.fit(X_train, y_train)

print("Train Accuracy is: ", lr.score(X_train, y_train))
print("Test Accuracy is: ", lr.score(X_test, y_test))

Train Accuracy is:  0.9030898876404494
Test Accuracy is:  0.8324022346368715


# Testing the model

In [12]:
test_data = pd.read_csv('./test.csv')
test_data.index = test_data['PassengerId']

In [13]:
test_data = create_Fszie_feature(test_data)

NameError: name 'create_Fszie_feature' is not defined

In [ ]:
#Encoding Categorical Variables
test_data = sex_embark_encode(test_data)

#### Check the encoding to be the same in both train & test sets

In [ ]:
print("male is encoded in train data as: ",train_data['Sex'][1])
print("female is encoded in train data as: ",train_data['Sex'][2])

print("male is encoded in test data as: ",test_data['Sex'][892])
print("female is encoded in test data as: ",test_data['Sex'][893])

In [ ]:
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].dropna().mean()) # for test data

mean_val= test_data['Age'].mean()
test_data['Age']=test_data['Age'].fillna(mean_val)
features_toDrop = ['Ticket', 'Cabin', 'Name', 'PassengerId']

test_data = test_data.drop(features_toDrop, axis = 1)

In [ ]:
Ids = test_data.index
test_data = sc.transform(test_data)

In [ ]:
def submit():
    Y = lr.predict(test_data)
    pred = {"PassengerId": Ids, "Survived": Y}
    pred = pd.DataFrame(pred)
    pred.to_csv('Subb.csv', index = False, encoding = 'utf-8')

In [ ]:
submit()